# M2 Bump Test

This Jupyter notebook is to run the bump test of M2.
Each axial actuator will be issued with a fixed force with pull/push movement one by one.
The target is to understand the feedback of neighboring actuators after the applying of force on a specific actuator.

## Import Modules

In [2]:
import asyncio
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

## Declaration of User-defined Functions

In [3]:
async def run_bump_test(csc, force, sleep_time=5.0):
    """Run the bump test.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    force : float
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Print the information of inclinometer
    zenithAngle = csc.tel_zenithAngle.get()
    print(f"Measured zenith angle: {zenithAngle.measured:0.2f} degree.")
    print(f"Raw inclinometer angle: {zenithAngle.inclinometerRaw:0.2f} degree.")
    print(f"Processed inclinometer angle: {zenithAngle.inclinometerProcessed:0.2f} degree.")

    for idx in range(NUM_ACTUATOR - NUM_TANGENT_LINK):
        print(f"Do the bump test of actuator: {idx}.")
        await bump_axial_actuator(csc, idx, force, sleep_time=sleep_time)

In [4]:
async def bump_axial_actuator(csc, idx_actuator, force, sleep_time=5):
    """Bump the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    idx_actuator : int
        Index of actuator.
    force : float
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Do the positive direction first
    num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
    forces = [0] * num_axial_actuator
    forces[idx_actuator] = abs(force)

    print(f"Apply the force: {abs(force)} N.")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)

    # Do the Negative direction
    forces[idx_actuator] = -abs(force)

    print(f"Apply the force: {-abs(force)} N.")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)

## Prepare the M2 CSC and put to Enabled state

In [5]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [6]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=10)

## Do the Bump Test

In [7]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

UTC time to is 2023-05-22 21:55:37.964906 now.


In [8]:
force = 10
await run_bump_test(m2, force, sleep_time=5)

Measured zenith angle: -0.09 degree.
Raw inclinometer angle: 90.09 degree.
Processed inclinometer angle: 88.97 degree.
Do the bump test of actuator: 0.
Apply the force: 10 N.
Reset the force.
Apply the force: -10 N.
Reset the force.
Do the bump test of actuator: 1.
Apply the force: 10 N.
Reset the force.
Apply the force: -10 N.
Reset the force.
Do the bump test of actuator: 2.
Apply the force: 10 N.
Reset the force.
Apply the force: -10 N.
Reset the force.
Do the bump test of actuator: 3.
Apply the force: 10 N.
Reset the force.
Apply the force: -10 N.
Reset the force.
Do the bump test of actuator: 4.
Apply the force: 10 N.
Reset the force.
Apply the force: -10 N.
Reset the force.
Do the bump test of actuator: 5.
Apply the force: 10 N.
Reset the force.
Apply the force: -10 N.
Reset the force.
Do the bump test of actuator: 6.
Apply the force: 10 N.
Reset the force.
Apply the force: -10 N.
Reset the force.
Do the bump test of actuator: 7.
Apply the force: 10 N.
Reset the force.
Apply the 